In [1]:
import tensorflow as tf
import numpy as np
from absl import flags,app
import copy

2023-04-05 13:49:14.925585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 13:49:15.008510: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-05 13:49:15.008523: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-05 13:49:15.444695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [13]:
pred = np.array([[2,4,6],[2,4,6]])
obs = np.array([[[1,2,3],[1,2,3]],[[1,2,3],[1,2,3]]])
act = np.array([[[1,2,3],[1,2,3]]])

In [14]:
print(obs.shape)
print(pred.shape)
print(act.shape)

(2, 2, 3)
(2, 3)
(1, 2, 3)


In [15]:
act - pred

array([[[-1, -2, -3],
        [-1, -2, -3]]])

In [16]:
len(obs.shape)

3

In [17]:
np.array([pred]).shape

(1, 2, 3)

In [18]:
file = 'A4056_weather.csv'

In [19]:
file[:-4]

'A4056_weather'

In [2]:
from invclass.utils import load_data, WindowGenerator

In [3]:
from invclass.inv_utils import inv_gradient, inv_gradient_ind, set_parameters, save_result, set_bounds, obj_fun

In [4]:
from invclass.proj_simplex import proj_simplex

In [5]:
FLAGS = flags.FLAGS
flags.DEFINE_string('data_path', '', 'Path to the data. Required.')
flags.DEFINE_string('data_file', '', 'Name of the file containing the data. Must be \
                   a pickle file. Required.')
flags.DEFINE_string('model_file', '', 'File containing inverse class model. Required.')
flags.DEFINE_string('ind_model_file', '', 'File containing the ind model. Required.')
flags.DEFINE_string('save_file', 'invResult.pkl', 'File name to save results to. Saves to data_path.')
flags.DEFINE_float('budget_start',1,'Starting budget for inverse classification. Default: 1')
flags.DEFINE_float('budget_end',10, 'Ending budget for inverse classification. Set budget_end =\
                     budget_start if only one budget value is desired. Default: 10')
flags.DEFINE_float('budget_interval',1,'Amount by which to increase the budget values from budget_start\
                   to budget_end. Default: 1')
flags.DEFINE_integer('max_iters', 50, 'Maximum number of gradient descent iterations. Default: 100')
flags.DEFINE_float('grad_tol', .0001, 'Gradient descent stopping criteria (\epsilon). Default: .0001')
flags.DEFINE_float('lam', 10, 'Initial gradient multiplier. Default: 10')
flags.DEFINE_integer('input_width', 6, 'Number of time steps of the input window. Default: 6')
flags.DEFINE_integer('label_width', 6, 'Number of time steps of the label window. Default: 6')
flags.DEFINE_integer('shift', 1, 'Time offset between input window and label window. Default: 1')
_ = FLAGS(["__main__"])

In [6]:
DATA_PATH = './brazil_data/'
DATA_FILE = 'processed_brazil.pkl'
MODEL_FILE = 'reg_model.h5'
IND_MODEL_FILE = 'ind_model.h5'
INPUT_WIDTH = 6
LABEL_WIDTH = 6
SHIFT = 1

In [7]:
data_dict = load_data(DATA_PATH, DATA_FILE, file_type='pkl')

In [19]:
data_dict['train']['X'].shape

(800, 25)

In [23]:
np.max([0,0])

0

In [7]:
reg_model = tf.keras.models.load_model(DATA_PATH+MODEL_FILE)
ind_model = tf.keras.models.load_model(DATA_PATH+IND_MODEL_FILE)

In [8]:
inv_data = data_dict['test']
X_inv = inv_data['X']
X_ids = inv_data['ids']

In [242]:
param_dict = set_parameters(data_dict)

In [10]:
window = WindowGenerator(
    input_width = INPUT_WIDTH,
    label_width = LABEL_WIDTH,
    shift = SHIFT,
    data_dict = data_dict
)

In [11]:
[(inputs, labels)] = window.test

In [248]:
inputs[1]

<tf.Tensor: shape=(6, 25), dtype=float64, numpy=
array([[ 0.958,  0.341,  0.297,  0.067,  0.75 ,  1.143, -0.23 ,  0.181,
         0.135,  0.917,  0.976,  0.227,  0.086, -0.019,  0.033,  0.181,
         0.698,  0.   ,  0.124,  0.417,  0.906,  0.137,  0.067,  0.284,
         0.604],
       [ 0.564,  0.837,  0.343,  0.146,  0.854,  1.145, -0.232,  0.124,
         0.135,  0.875,  0.898,  0.08 ,  0.   ,  0.019,  0.016,  0.181,
         0.698,  0.   ,  0.143,  0.177,  0.819,  0.137,  0.117,  0.284,
         0.604],
       [ 0.341,  0.484,  0.427,  0.25 ,  0.933,  1.146, -0.235,  0.133,
         0.154,  0.743,  0.661,  0.307,  0.186,  0.302,  0.131,  0.181,
         0.698,  0.   ,  0.152,  0.009,  0.601,  0.356,  0.367,  0.284,
         0.604],
       [ 0.558,  0.595,  0.645,  0.371,  0.983,  1.147, -0.237,  0.181,
         0.163,  0.576,  0.583,  0.307,  0.171,  0.321,  0.279,  0.181,
         0.698,  0.   ,  0.2  ,  0.402,  0.529,  0.178,  0.333,  0.284,
         0.604],
       [ 0.391,  0.

In [194]:
inv_inds = list(range(inputs.shape[0]))

In [13]:
grads = inv_gradient(reg_model, inputs, labels)

INV_GRADIENT


In [14]:
nz_grads = np.nonzero(grads)

In [15]:
nz_inds = list(set(nz_grads[0]))

In [16]:
print("Total test instances w/ non-zero grads: {}".format(len(nz_inds)),
           "out of {} total instances".format(grads.shape[0]))

Total test instances w/ non-zero grads: 59 out of 59 total instances


In [207]:
result_dict = {"budgets":param_dict['budgets'],'ids':[]}
improv_mat = np.zeros((len(inv_inds),len(param_dict['budgets'])+1))

In [208]:
idv = 0

In [19]:
inv_inputs = inputs[idv].numpy()
inv_labels = labels[idv].numpy()

In [20]:
budgets = param_dict['budgets']
index_dict = param_dict['inds']
xU_i = index_dict['xU_ind']
xI_i = index_dict['xI_ind']
xD_i = index_dict['xD_ind']
xD_ii = index_dict['xD_ind_ind']

In [21]:
xU_xD = np.hstack([inv_inputs[:,xU_i], inv_inputs[:,xD_i]])

In [22]:
xI_est = ind_model.predict(xU_xD)

1/1 [==============================] - 0s 68ms/step


In [23]:
x_init = np.array([ np.hstack([inv_inputs[:,xU_i], xI_est, inv_inputs[:,xD_i]]) ])

In [25]:
y_hat_init = reg_model.predict(x_init)[0]

1/1 [==============================] - 0s 437ms/step


In [28]:
obj_val_init = obj_fun(reg_model, x_init, inv_labels)

In [29]:
xD_opt_mat = np.zeros((len(budgets)+1, inv_inputs.shape[0], len(xD_i)), dtype=np.float32)
xI_opt_mat = np.zeros((len(budgets)+1, inv_inputs.shape[0], len(xI_i)), dtype=np.float32)
opt_obj_vect = np.zeros((len(budgets)+1), dtype=np.float32)

In [30]:
xD_opt_mat[0] = inv_inputs[:,xD_i]
xI_opt_mat[0] = xI_est
opt_obj_vect[0] = obj_val_init

In [31]:
reg_grad_full = inv_gradient(reg_model, x_init, inv_labels)[0]
ind_grad_full = inv_gradient_ind(ind_model, xU_xD, num_loss=len(xI_i))

INV_GRADIENT
INV_GRADIENT_IND


In [33]:
xD_grad = reg_grad_full[:,xD_i]
xI_grad = reg_grad_full[:,xI_i]
xD_ind_grad = ind_grad_full[:,xD_ii]

Convert to np.float16 because np.float32 breaks np.matmul

In [34]:
xD_grad = xD_grad.astype(np.float16)
xI_grad = xI_grad.astype(np.float16)
xD_ind_grad = xD_ind_grad.astype(np.float16)

In [35]:
opt_grad = xD_grad + np.matmul(xI_grad, xD_ind_grad)

In [37]:
d, c, l, u = set_bounds(inv_inputs, -1*opt_grad, param_dict)

In [65]:
bud_iter = 1

In [66]:
b = budgets[0]

In [67]:
bud_iter += 1

In [68]:
diff = np.inf
tot_iters = 0

In [73]:
if bud_iter == 1:
    prev_xD = copy.deepcopy(inv_inputs[:,xD_i])
    prev_xI = xI_est
else:
    prev_xD = xD_opt_mat[0]
    prev_xI = xI_opt_mat[0]

In [78]:
opt_xD = copy.deepcopy(prev_xD)

In [83]:
full_opt_x = np.array([ np.hstack([inv_inputs[:,xU_i], prev_xI, opt_xD]) ])
xU_xD_opt = np.hstack([inv_inputs[:,xU_i], opt_xD])

In [86]:
gStep = FLAGS.lam

In [89]:
obj_vect = [opt_obj_vect[0]]

In [99]:
reg_grad_full = inv_gradient(reg_model, full_opt_x, inv_labels)[0]
ind_grad_full = inv_gradient_ind(ind_model, xU_xD_opt, num_loss=len(xI_i))

INV_GRADIENT
INV_GRADIENT_IND


In [100]:
xD_grad = reg_grad_full[:,xD_i]
xI_grad = reg_grad_full[:,xI_i]
xD_ind_grad = ind_grad_full[:,xD_ii]

In [101]:
xD_grad = xD_grad.astype(np.float16)
xI_grad = xI_grad.astype(np.float16)
xD_ind_grad = xD_ind_grad.astype(np.float16)

In [102]:
opt_grad = xD_grad + np.matmul(xI_grad,xD_ind_grad)

In [103]:
temp_opt_xD = opt_xD - 1/gStep*opt_grad

In [104]:
temp_opt_xD_diff = np.multiply(d,temp_opt_xD - prev_xD)

In [120]:
proj_xD_diff = []

for r_temp_opt_xD_diff, r_c, r_l, r_u in zip(temp_opt_xD_diff,c,l,u):
    proj_xD_diff.append( proj_simplex(r_temp_opt_xD_diff,b,r_c,r_l,r_u) )

proj_xD_diff = np.array(proj_xD_diff)

In [121]:
proj_xD_diff = np.multiply(d,proj_xD_diff)

In [122]:
opt_xD = prev_xD + proj_xD_diff

In [123]:
xU_xD_opt = np.hstack([inv_inputs[:,xU_i], opt_xD])
xI_est = ind_model.predict(xU_xD_opt)

1/1 [==============================] - 0s 18ms/step


In [128]:
full_opt_x = np.array([ np.hstack([inv_inputs[:,xU_i], xI_est, opt_xD]) ])

In [131]:
cObj = obj_fun(reg_model, full_opt_x, inv_labels)

In [141]:
gStep *= 2

In [145]:
reg_grad_full = inv_gradient(reg_model, full_opt_x, inv_labels)[0]
ind_grad_full = inv_gradient_ind(ind_model, xU_xD_opt, num_loss=len(xI_i))

INV_GRADIENT
INV_GRADIENT_IND


In [146]:
xD_grad = reg_grad_full[:,xD_i]
xI_grad = reg_grad_full[:,xI_i]
xD_ind_grad = ind_grad_full[:,xD_ii]

In [147]:
xD_grad = xD_grad.astype(np.float16)
xI_grad = xI_grad.astype(np.float16)
xD_ind_grad = xD_ind_grad.astype(np.float16)

In [148]:
opt_grad = xD_grad + np.matmul(xI_grad,xD_ind_grad)

In [149]:
temp_opt_xD = opt_xD - 1/gStep*opt_grad
temp_opt_xD_diff = np.multiply(d,temp_opt_xD - prev_xD)

In [150]:
proj_xD_diff = []

for r_temp_opt_xD_diff, r_c, r_l, r_u in zip(temp_opt_xD_diff,c,l,u):
    proj_xD_diff.append( proj_simplex(r_temp_opt_xD_diff,b,r_c,r_l,r_u) )

proj_xD_diff = np.array(proj_xD_diff)

In [151]:
proj_xD_diff = np.multiply(d,proj_xD_diff)

In [152]:
opt_xD = prev_xD + proj_xD_diff

In [153]:
xU_xD_opt = np.hstack([inv_inputs[:,xU_i], opt_xD])
xI_est = ind_model.predict(xU_xD_opt)

1/1 [==============================] - 0s 16ms/step


In [154]:
full_opt_x = np.array([ np.hstack([inv_inputs[:,xU_i], xI_est, opt_xD]) ])

In [158]:
cObj = obj_fun(reg_model, full_opt_x, inv_labels).numpy()

In [159]:
obj_vect.append(cObj)

In [163]:
diff = (obj_vect[-2]-obj_vect[-1])/obj_vect[-2]

In [165]:
gStep = gStep/1.5

In [172]:
xD_opt_mat[bud_iter] = opt_xD
xI_opt_mat[bud_iter] = xI_est

In [175]:
if obj_vect[-1] > obj_vect[-2]:
    opt_obj_vect[bud_iter] = obj_vect[-2]
else:
    opt_obj_vect[bud_iter] = obj_vect[-1]

In [176]:
opt_obj_vect

array([0.9861606, 0.       , 0.9774654, 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       ],
      dtype=float32)

In [214]:
return_dict={
    "obj":opt_obj_vect,
    "xD":xD_opt_mat,
    "xI":xI_opt_mat,
}

In [215]:
inv_dat = return_dict

In [216]:
result_dict['ids'].append(X_ids[idv])
result_dict[X_ids[idv]] = inv_dat

In [218]:
improv_mat[idv] = inv_dat['obj']

In [217]:
result_dict

{'budgets': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 'ids': [586],
 586: {'obj': array([0.9861606, 0.       , 0.9774654, 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.       , 0.       , 0.       ],
        dtype=float32),
  'xD': array([[[0.        , 0.53333336, 0.402311  , 0.442029  , 0.8630137 ,
           0.8       , 0.28377885, 0.60373044],
          [0.        , 0.5714286 , 0.402311  , 0.33333334, 0.7671233 ,
           0.8666667 , 0.28377885, 0.60373044],
          [0.        , 0.552381  , 0.402311  , 0.29710144, 0.72602737,
           0.9       , 0.28377885, 0.60373044],
          [0.        , 0.52380955, 0.402311  , 0.26811594, 0.7123288 ,
           0.9166667 , 0.28377885, 0.60373044],
          [0.        , 0.50476193, 0.402311  , 0.21014492, 0.65753424,
           0.96666664, 0.28377885, 0.60373044],
          [0.        , 0.45714286, 0.402311  , 0.20289855, 0.6438356 ,
           0.96666664, 0.28377885, 0.60373044]],
  
         [[0.       

In [224]:
avg_opt = np.mean(improv_mat, axis=0)

In [226]:
budgets = param_dict['budgets']
budgets.insert(0,0)

In [230]:
res_mat = np.vstack([budgets, avg_opt])

In [233]:
np.set_printoptions(precision=3)
print("Average probability by budget:\n {}".format(res_mat))

Average probability by budget:
 [[ 0.     1.     2.     3.     4.     5.     6.     7.     8.     9.
  10.   ]
 [ 0.017  0.     0.017  0.     0.     0.     0.     0.     0.     0.
   0.   ]]


In [232]:
save_result(result_dict)

In [240]:
param_dict['budgets']

[0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [241]:
budgets

[0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [249]:
budgets = param_dict['budgets']
budgets.insert(0,0)
budget_inputs = []
for b in budgets:
    budget_inputs.append(copy.deepcopy(inputs))
budget_inputs = np.array(budget_inputs)

In [276]:
budget_inputs.shape

(11, 59, 6, 25)

In [286]:
budget_inputs[:,1,:-1,:][:,:,xI_ind].shape

(11, 5, 10)

In [257]:
x= budget_inputs[:,0]

In [258]:
x[0].shape

(6, 25)

In [263]:
inv_dat = result_dict[586]
idv = 0

In [266]:
xD_ind = data_dict['xD_ind']
xI_ind = data_dict['xI_ind']

In [287]:
budget_inputs[:,idv+1,:-1,:][:,:,xI_ind] = inv_dat['xI'][:,1:,:]
budget_inputs[:,idv+1,:-1,:][:,:,xD_ind] = inv_dat['xD'][:,1:,:]

In [269]:
budget_inputs[:,idv+1,:-1,xI_ind].shape

(10, 11, 5)

In [288]:
budget_inputs

array([[[[ 0.47 ,  0.498,  0.604, ...,  0.8  ,  0.284,  0.604],
         [ 0.266,  0.392,  0.267, ...,  0.867,  0.284,  0.604],
         [ 0.337,  0.47 ,  0.452, ...,  0.9  ,  0.284,  0.604],
         [ 0.628,  0.553,  0.817, ...,  0.917,  0.284,  0.604],
         [ 0.725,  0.059,  0.508, ...,  0.967,  0.284,  0.604],
         [ 0.516,  0.482,  0.395, ...,  0.967,  0.284,  0.604]],

        [[ 0.958,  0.341,  0.297, ...,  0.867,  0.284,  0.604],
         [ 0.564,  0.837,  0.343, ...,  0.9  ,  0.284,  0.604],
         [ 0.341,  0.484,  0.427, ...,  0.917,  0.284,  0.604],
         [ 0.558,  0.595,  0.645, ...,  0.967,  0.284,  0.604],
         [ 0.391,  0.335,  0.368, ...,  0.967,  0.284,  0.604],
         [ 0.525,  0.399,  0.547, ...,  0.367,  0.284,  0.604]],

        [[ 0.593,  0.461,  0.422, ...,  0.5  ,  0.284,  0.604],
         [ 0.131,  0.311,  0.71 , ...,  0.65 ,  0.284,  0.604],
         [ 0.457,  0.561,  0.393, ...,  0.767,  0.284,  0.604],
         [ 0.739,  0.587,  0.541, ..

In [271]:
inv_dat['xI'][:,1:,:].shape

(11, 5, 10)

In [290]:
return_dict['obj'].shape

(11,)